In [11]:
import pandas as pd
import os
import geopandas as gpd
from keplergl import KeplerGl
import json


In [3]:
morbi = pd.read_csv('../data/morbilidad_Fgen.csv')

In [4]:
morbi.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
21172,4,1,4,2,F1229,1,34,13,1.00000,F12,...,2016-04-13,2016-03-31,2016,3,31,Urgente,Curación,Hombre,Almería,Almería
5953,8,2,8,2,F05,2,83,43,1.00504,F05,...,2016-04-19,2016-03-07,2016,3,7,Urgente,Traslado,Mujer,Barcelona,Barcelona
323462,43,2,43,2,F609,1,39,3,1.02709,F60,...,2018-02-12,2018-02-09,2018,2,9,Urgente,Curación,Mujer,Tarragona,Tarragona


In [5]:
morbi.fecha_ing =  pd.to_datetime(morbi.fecha_ing)
morbi.fecha_al =  pd.to_datetime(morbi.fecha_al)

In [6]:
morbi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438929 entries, 0 to 438928
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   ProvHosp      438929 non-null  int64         
 1   Sexo          438929 non-null  int64         
 2   ProvResi      438929 non-null  int64         
 3   DiagEntr      438929 non-null  int64         
 4   DiagPrin      438929 non-null  object        
 5   MotivoAlta    438929 non-null  int64         
 6   Edad          438929 non-null  int64         
 7   Dias          438929 non-null  int64         
 8   Factor        438929 non-null  float64       
 9   code          438929 non-null  object        
 10  cap           438929 non-null  object        
 11  fecha_al      438929 non-null  datetime64[ns]
 12  fecha_ing     438929 non-null  datetime64[ns]
 13  year          438929 non-null  int64         
 14  month         438929 non-null  int64         
 15  day           438

In [7]:
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

In [45]:
# #Create a basemap 
# map_0= KeplerGl(height=700, weight = 500, data={'morbilidad': morbi})
# #show the map
# map_0

# map_0.add_data(data=provincias, name='geojson')

In [48]:
total_pro = morbi.groupby('ProvHosp_nom').agg({'SexNom':'count'}).reset_index().rename({'SexNom':'ingresos'},axis=1)

In [49]:
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

In [8]:
poly = {p['properties']['name']:p['geometry'] for p in provincias['features'] }

In [9]:
total_pro['pol'] = total_pro['ProvHosp_nom'].map(poly)

NameError: name 'total_pro' is not defined

In [52]:
total_pro.sample()

,ProvHosp_nom,ingresos,pol
35,Palencia,2313,"{'type': 'MultiPolygon', 'coordinates': [[[[-4..."


In [53]:
total_pob = pd.read_csv('../data/poblacion_prov.csv')

In [54]:
total_pob = total_pob[total_pob['Sexo'] == 'Total']

In [55]:
total_pob.sample(3)

,Provincias,cod_pr,Sexo,Total
17,Badajoz,6,Total,676522.92
134,Toledo,45,Total,633374.67
65,Huesca,22,Total,217977.79


In [56]:
dic_pob = {}
for i,r in total_pob.iterrows():
    dic_pob[r.Provincias] = r.Total

In [57]:
total_pro['poblacion'] = total_pro.ProvHosp_nom.map(dic_pob)

In [58]:
total_pro.sample(3)

,ProvHosp_nom,ingresos,pol,poblacion
42,Tarragona,9281,"{'type': 'MultiPolygon', 'coordinates': [[[[1....",730851.79
22,Huesca,2685,"{'type': 'MultiPolygon', 'coordinates': [[[[-0...",217977.79
13,Cuenca,1327,"{'type': 'MultiPolygon', 'coordinates': [[[[-2...",205901.75


In [59]:
total_pro['ponde'] = ((total_pro.ingresos/total_pro.poblacion)*100000).round(2)

In [60]:
total_pro.sample(3)

,ProvHosp_nom,ingresos,pol,poblacion,ponde
40,Sevilla,12849,"{'type': 'MultiPolygon', 'coordinates': [[[[-5...",1856573.58,692.08
50,Álava,3874,"{'type': 'MultiPolygon', 'coordinates': [[[[-3...",308815.21,1254.47
33,Navarra,6973,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",607095.88,1148.58


In [61]:
#Create a basemap 
map_1= KeplerGl(height=700, weight = 500, data={'total_pro': total_pro})
#show the map
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'total_pro':               ProvHosp_nom  ingresos  \
0                 A Coruña      7899   
1 …

In [62]:
morbi.sample()

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
78113,28,1,28,2,F333,1,69,13,1.08548,F33,...,2016-05-10,2016-04-27,2016,4,27,Urgente,Curación,Hombre,Madrid,Madrid


In [63]:
pro_day = morbi.groupby(['ProvHosp_nom','year','month']).agg({'day':'count','fecha_ing':'min'}).reset_index().rename({'day':'total'},axis=1)

In [64]:
pro_day.sample()

,ProvHosp_nom,year,month,total,fecha_ing
477,Baleares,2014,12,5,2014-12-03


In [65]:
pro_day.shape

(3997, 5)

In [66]:
pro_day['geometry']= pro_day.ProvHosp_nom.map(poly)

In [ ]:
map_2= KeplerGl(height=700, weight = 700, data={'Ingresos_dia': pro_day})
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
